[Reference](https://medium.com/@dmitry.romanoff/automating-database-health-checks-with-python-a-step-by-step-guide-1382b3108e97)

# 1. Loading Configuration

In [1]:
import yaml

def load_config(config_file):
    with open(config_file, 'r') as file:
        return yaml.safe_load(file)

# 2. Connecting to the Database

In [2]:
import psycopg2

def connect_db(hostname, port, dbname, username, password):
    try:
        conn = psycopg2.connect(
            host=hostname,
            port=port,
            dbname=dbname,
            user=username,
            password=password
        )
        return conn
    except Exception as e:
        print(f"Error connecting to database: {e}")
        raise

# 3. Executing Queries

In [3]:
def execute_queries(conn, queries):
    results = []
    try:
        with conn.cursor() as cursor:
            for check in queries:
                description = list(check.keys())[0]
                query = check[description]['query']
                cursor.execute(query)
                result = cursor.fetchall()
                columns = [desc[0] for desc in cursor.description]  # Get column names
                results.append({
                    'description': check[description]['description'],
                    'query': query,
                    'columns': columns,
                    'result': result
                })
    except Exception as e:
        print(f"Error executing queries: {e}")
        raise
    return results

# 4. Generating HTML Report

In [4]:
from jinja2 import Template

def generate_html_report(results, output_file):
    template = Template("""
    <!DOCTYPE html>
    <html lang="en">
    <head>
        <meta charset="UTF-8">
        <meta name="viewport" content="width=device-width, initial-scale=1.0">
        <title>Database Check Results</title>
        <style>
            table {
                width: 100%;
                border-collapse: collapse;
            }
            table, th, td {
                border: 1px solid black;
            }
            th, td {
                padding: 8px;
                text-align: left;
            }
            th {
                background-color: #f2f2f2;
            }
            pre {
                white-space: pre-wrap; /* Allows wrapping of long queries */
            }
        </style>
    </head>
    <body>
        <h1>Database Check Results</h1>
        {% for check in results %}
        <h2>{{ check.description }}</h2>
        <pre>Query: {{ check.query }}</pre>
        <h3>Result:</h3>
        <table>
            <thead>
                <tr>
                    {% for column in check.columns %}
                    <th>{{ column }}</th>
                    {% endfor %}
                </tr>
            </thead>
            <tbody>
                {% for row in check.result %}
                <tr>
                    {% for value in row %}
                    <td>{{ value }}</td>
                    {% endfor %}
                </tr>
                {% endfor %}
            </tbody>
        </table>
        {% endfor %}
    </body>
    </html>
    """)
    html_content = template.render(results=results)
    with open(output_file, 'w') as file:
        file.write(html_content)

In [5]:
def main():
    # Database parameters
    db_hostname = 'localhost'
    db_port = 5439
    db_name = 'my_database'
    db_username = 'my_user'
    db_pwd = 'my_password'

# File paths
    config_file = 'dbcheck_config.yaml'
    output_file = 'result.html'
    # Load configuration
    config = load_config(config_file)
    dbchecks = config['dbchecks']
    # Connect to the database
    conn = connect_db(db_hostname, db_port, db_name, db_username, db_pwd)
    try:
        # Execute queries and get results
        results = execute_queries(conn, dbchecks)
        # Generate HTML report
        generate_html_report(results, output_file)
    finally:
        # Close the database connection
        conn.close()
if __name__ == "__main__":
    main()